In [1]:
pip install surprise

  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp38-cp38-win_amd64.whl size=719518 sha256=56cd782ccce721a782f95971898b889c0d76bc416c0b884c5ddd1b67a8d365c4
  Stored in directory: c:\users\x\appdata\local\pip\cache\wheels\20\91\57\2965d4cff1b8ac7ed1b6fa25741882af3974b54a31759e10b6
Successfully built scikit-surprise
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from surprise import SVD
from surprise import KNNBasic
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

Config name is missing.
Please pick one among the available configs: ['Wireless_v1_00', 'Watches_v1_00', 'Video_Games_v1_00', 'Video_DVD_v1_00', 'Video_v1_00', 'Toys_v1_00', 'Tools_v1_00', 'Sports_v1_00', 'Software_v1_00', 'Shoes_v1_00', 'Pet_Products_v1_00', 'Personal_Care_Appliances_v1_00', 'PC_v1_00', 'Outdoors_v1_00', 'Office_Products_v1_00', 'Musical_Instruments_v1_00', 'Music_v1_00', 'Mobile_Electronics_v1_00', 'Mobile_Apps_v1_00', 'Major_Appliances_v1_00', 'Luggage_v1_00', 'Lawn_and_Garden_v1_00', 'Kitchen_v1_00', 'Jewelry_v1_00', 'Home_Improvement_v1_00', 'Home_Entertainment_v1_00', 'Home_v1_00', 'Health_Personal_Care_v1_00', 'Grocery_v1_00', 'Gift_Card_v1_00', 'Furniture_v1_00', 'Electronics_v1_00', 'Digital_Video_Games_v1_00', 'Digital_Video_Download_v1_00', 'Digital_Software_v1_00', 'Digital_Music_Purchase_v1_00', 'Digital_Ebook_Purchase_v1_00', 'Camera_v1_00', 'Books_v1_00', 'Beauty_v1_00', 'Baby_v1_00', 'Automotive_v1_00', 'Apparel_v1_00', 'Digital_Ebook_Purchase_v1_01', 'Books_v1_01', 'Books_v1_02']
Example of usage:
	`load_dataset('amazon_us_reviews', 'Wireless_v1_00')`


In [3]:
from datasets import load_dataset
dataset = load_dataset("amazon_us_reviews", 'Books_v1_00')

Reusing dataset amazon_us_reviews (C:\Users\X\.cache\huggingface\datasets\amazon_us_reviews\Books_v1_00\0.1.0\17b2481be59723469538adeb8fd0a68b0ba363bbbdd71090e72c325ee6c7e563)


In [4]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', 'product_title', 'product_category', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'review_headline', 'review_body', 'review_date'],
        num_rows: 10319090
    })
})


In [5]:
dataset['train'][0]

{'marketplace': 'US',
 'customer_id': '25933450',
 'review_id': 'RJOVP071AVAJO',
 'product_id': '0439873800',
 'product_parent': '84656342',
 'product_title': 'There Was an Old Lady Who Swallowed a Shell!',
 'product_category': 'Books',
 'star_rating': 5,
 'helpful_votes': 0,
 'total_votes': 0,
 'vine': 0,
 'verified_purchase': 1,
 'review_headline': 'Five Stars',
 'review_body': 'I love it and so does my students!',
 'review_date': '2015-08-31'}

In [6]:
df = pd.DataFrame(dataset['train']['customer_id'])
df['product_id'] = dataset['train']['product_id']
df['star_rating'] = dataset['train']['star_rating']
df = df.rename(columns={0: "customer_id"})

In [17]:
df = df.rename(columns={'star_rating': "raw_ratings"})

In [18]:
df

,customer_id,product_id,raw_ratings
0,25933450,0439873800,5
1,1801372,1623953553,5
2,5782091,142151981X,5
3,32715830,014241543X,5
4,14005703,1604600527,5
...,...,...,...
10319085,52004451,B003ZX1EQS,5
10319086,36099005,0786652918,4
10319087,38081951,1608981215,4
10319088,53040283,0670023418,5


In [29]:
pip install plotly


Note: you may need to restart the kernel to use updated packages.


In [31]:
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

data = df['raw_ratings'].value_counts().sort_index(ascending=False)
trace = go.Bar(x = data.index,
               text = ['{:.1f} %'.format(val) for val in (data.values / df.shape[0] * 100)],
               textposition = 'auto',
               textfont = dict(color = '#000000'),
               y = data.values,
               )
# Create layout
layout = dict(title = 'Distribution Of {} raw-ratings'.format(df.shape[0]),
              xaxis = dict(title = 'Rating'),
              yaxis = dict(title = 'Count'))
# Create plot
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

In [20]:
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(df[['customer_id', 'product_id', 'raw_ratings']], reader)

In [21]:
%%time
trainset=data.build_full_trainset()

CPU times: total: 35.7 s
Wall time: 35.8 s


In [22]:
%%time
testset=trainset.build_testset()

CPU times: total: 11.6 s
Wall time: 11.6 s


In [27]:
algo =SVD()
algo.fit(trainset)

In [23]:


from surprise.model_selection import GridSearchCV
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(trainset)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

AttributeError: 'Trainset' object has no attribute 'raw_ratings'